In [1]:
import numpy as np
import pandas as pd
from PIL import Image as pilImg
import os 
import cv2
from datetime import datetime
import matplotlib.pyplot as plt
import itertools

1. Підготовка Даних із завантаженого датасета із Англійскими словами
Source: https://www.robots.ox.ac.uk/~vgg/data/text/#sec-synth

Із file_path(шлях до txt файлу із описами картинок) отримує картинки у кількості number
Повертає тільки картинки з текстом розміром від 4 до 12 елементів

Takes the file path of images annotation txt file with the number of images names to be extracted
and returns the list of file names having label length <=12

In [6]:
def Extract_image_names(file_path, number):
    with open(file_path) as file:
        files = file.readlines()
        file.close()
        count = 0
        img_names = []
        for f in files:
            """
            Extract from 'SynthImageDataset./2425/1/115_Lube_45484.jpg 45484' only image lable - Lube
            """
            label = f.split('_')[1]
            if len(label) >= 4 and len(label) <= 12:
                img_names.append(f)
                count += 1
            if count == number:
                break
        images_names=['SynthImageDataset' + x.strip() for x in img_names]
        return images_names

In [7]:
train_images = Extract_image_names('SynthImageDataset/annotation_train.txt',200000)
train_images[:5]

['SynthImageDataset./2425/1/115_Lube_45484.jpg 45484',
 'SynthImageDataset./2425/1/114_Spencerian_73323.jpg 73323',
 'SynthImageDataset./2425/1/112_CARPENTER_11682.jpg 11682',
 'SynthImageDataset./2425/1/110_savannas_67969.jpg 67969',
 'SynthImageDataset./2425/1/109_unfix_82473.jpg 82473']

Очищує назву файлів забираючи із кінця цифри, що індикують номер картинки

In [11]:
#помістити всередину минулої функції
def clean_file_names(file_names):
    clean_files = []
    for file in file_names:
        main_folder, img_loc, extension = file.split('.')
        extension = extension.split(' ')[0]
        img_file = main_folder + img_loc + '.' + extension
        clean_files.append(img_file)
    return clean_files


train_cleaned = clean_file_names(train_images)

train_cleaned[:5]

['SynthImageDataset/2425/1/115_Lube_45484.jpg',
 'SynthImageDataset/2425/1/114_Spencerian_73323.jpg',
 'SynthImageDataset/2425/1/112_CARPENTER_11682.jpg',
 'SynthImageDataset/2425/1/110_savannas_67969.jpg',
 'SynthImageDataset/2425/1/109_unfix_82473.jpg']

In [12]:
train_data=pd.DataFrame({'ImageName':train_cleaned})
train_data.head()

,ImageName
0,SynthImageDataset/2425/1/115_Lube_45484.jpg
1,SynthImageDataset/2425/1/114_Spencerian_73323.jpg
2,SynthImageDataset/2425/1/112_CARPENTER_11682.jpg
3,SynthImageDataset/2425/1/110_savannas_67969.jpg
4,SynthImageDataset/2425/1/109_unfix_82473.jpg
